In [2]:
import csv
import math

import matplotlib.pyplot as plt
import numpy as np

from keras import applications
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model


import cv2
from matplotlib import pyplot as plt
from skimage import color
from skimage.transform import resize

from IPython.display import Audio
from IPython.display import display
import time

Using TensorFlow backend.


In [3]:
import numpy as np

print(len(np.load('pac_arrays/train_all.npy')))
print(len(np.load('pac_arrays/test_all.npy')))
print(len(np.load('pac_arrays/dev_all.npy')))

25740
8580
8580


In [4]:
import csv
with open("pac_guide.csv") as file:
    reader = csv.reader(file)
    i = 0
    image_paths = []
    camera_nums = []
    labels = []
    
    for row in reader:
        index = row[0]
        camera_num = row[1]
        label = row[2]
        filename = row[3]
        image_paths.append(filename)
        camera_nums.append(camera_num)
        labels.append(label)
        #print(index, camera_num, label, filename)
print(len(image_paths)) # number of rows: 112036
print(len(camera_nums))

sub_dir_hash = {}
for i, sub_dir in enumerate(np.load('sub_dirs.npy')):
    sub_dir_hash[sub_dir] = i

print(sub_dir_hash)

112036
112036
{'04': 1, '72': 12, '11': 4, '39': 7, '23': 6, '59': 9, '06': 2, '15': 5, '62': 10, '02': 0, '08': 3, '63': 11, '52': 8}


In [5]:
counter1 = 0
counter0 = 0
for i in labels:
    if i == '1':
        counter1 += 1
    if i == '0':
        counter0 += 1
print(len(labels))
print(counter1)
print(counter0)

112036
11994
100042


In [6]:
def depth_map_to_image(depth_map):
    img = cv2.normalize(depth_map, depth_map, 0, 1, cv2.NORM_MINMAX)
    img = np.array(img * 255, dtype=np.uint8)
    orig_img = img
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.applyColorMap(img, cv2.COLORMAP_OCEAN)
    #img = resize(img, 224, 224)

    return orig_img, img

def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 2)) # Change to one-hot
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 2)) # Change to one-hot
            
            for i, index in enumerate(batch_indices):
                #img = image.load_img(image_paths[index], target_size=(224, 224))
                data = np.load(image_paths[int(index)])
                depth_map = data['x'].astype(np.float32)
                orig_img, ocean = depth_map_to_image(depth_map)
                ocean = resize(ocean, (224, 224), mode='constant')
                
                X[i, :, :, :] = ocean
                # Convert to 1 hot vector
                one_hot = np.zeros(2)
                #camera_num = camera_nums[int(index)]
                #camera_ind = sub_dir_hash[camera_num]
                if labels[int(index)] == '1':
                    one_hot[1] = 1
                elif labels[int(index)] == '0':
                    one_hot[0] = 1
                    
                #one_hot[camera_ind] = 1
                Y[i,:] = one_hot
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            
            yield (X, Y)

In [6]:
input_tensor = Input(shape=(224,224,3))
model = applications.VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)

57942016/58889256 [============================>.] - ETA: 0s

In [7]:
# build a classifier model to put on top of the convolutional model
x = model.output
x = Flatten(input_shape=(model.output_shape[1:]))(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(x)
x = Dense(2, activation='softmax', name='output', kernel_initializer='glorot_uniform')(x)

# add new classifier model on top of convolutional base
new_model = Model(model.input, x)

In [8]:
for layer in new_model.layers[:19]:
    layer.trainable = False

In [9]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [10]:
# RMSprop
new_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

In [11]:
minibatch_size = 64

train_indices = np.load('pac_arrays/train_all.npy')
test_indices = np.load('pac_arrays/test_all.npy')

np.random.shuffle(train_indices)
np.random.shuffle(test_indices)

image_generator(train_indices, minibatch_size)

epochs = 30
minibatch_size = 64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

class_weight = {0 : 1.,
    1: 7.}

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = class_weight,
    validation_data=image_generator(test_indices, minibatch_size),
    nb_val_samples=test_steps)

new_model.save('pca_camera_and_sanitation_classification.hdf5')



/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:27: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=30, class_weight={0: 1.0, 1..., steps_per_epoch=403, validation_data=<generator..., validation_steps=135)`


Epoch 1/30
403/403 [==============================] - 570s - loss: 1.5656 - acc: 0.5040 - val_loss: 1.1046 - val_acc: 0.1484
Epoch 2/30
403/403 [==============================] - 536s - loss: 1.3110 - acc: 0.5495 - val_loss: 0.6114 - val_acc: 0.6933
Epoch 3/30
403/403 [==============================] - 536s - loss: 1.2668 - acc: 0.5731 - val_loss: 1.8618 - val_acc: 0.1482
Epoch 4/30
403/403 [==============================] - 535s - loss: 1.2298 - acc: 0.5935 - val_loss: 0.9156 - val_acc: 0.3047
Epoch 5/30
403/403 [==============================] - 535s - loss: 1.2069 - acc: 0.6001 - val_loss: 1.0624 - val_acc: 0.2587
Epoch 6/30
403/403 [==============================] - 535s - loss: 1.1880 - acc: 0.6105 - val_loss: 1.0760 - val_acc: 0.2698
Epoch 7/30
403/403 [==============================] - 537s - loss: 1.1827 - acc: 0.6156 - val_loss: 0.5438 - val_acc: 0.7420
Epoch 8/30
403/403 [==============================] - 536s - loss: 1.1650 - acc: 0.6194 - val_loss: 1.6740 - val_acc: 0.1526


In [33]:
time.sleep(2)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))
time.sleep(2)
display(Audio(url=sound_file, autoplay=True))

for i in range(0, 3):
    time.sleep(2)
    display(Audio(url=sound_file, autoplay=True))

In [8]:
new_model = load_model('pca_camera_and_sanitation_classification.hdf5')


In [9]:
new_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

In [13]:
minibatch_size = 64

train_indices = np.load('pac_arrays/train_all.npy')
test_indices = np.load('pac_arrays/test_all.npy')

image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

num0 = 0
num1 = 0

for i in train_indices:
    i = int(i)
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
train_ratio = int(num0 / num1)

num0 = 0
num1 = 0
for i in test_indices:
    i = int(i)
    if labels[i] == '0':
        num0 += 1
    elif labels[i] == '1':
        num1 += 1
test_ratio = int(num0 / num1)


print(train_ratio, test_ratio)

train_weight = {0 : 1.,
1: train_ratio}

test_weight = {0 : 1.,
1: test_ratio}


5 5


In [14]:
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    class_weight = train_weight
    )

Epoch 1/1
403/403 [==============================] - 384s - loss: 0.8989 - acc: 0.7342   


In [11]:
history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    class_weight = test_weight
    )

Epoch 1/1
135/135 [==============================] - 137s - loss: 0.8921 - acc: 0.7334   
